In [15]:
!pip install google-search-results

Defaulting to user installation because normal site-packages is not writeable


In [16]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms.huggingface_hub import HuggingFaceHub
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

In [17]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers

In [19]:
from langchain.agents import AgentExecutor

In [20]:
import os

#os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
os.environ["SERPAPI_API_KEY"] = "c9ab8a0b29aa07ab3581e6d3258edadfa43c6c86d631d754527eaa54595b6d30"

In [21]:
from transformers import AutoTokenizer

model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)

In [22]:
import torch
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    max_length=500,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

ValueError: Could not load model tiiuae/falcon-7b-instruct with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.falcon.modeling_falcon.FalconForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/home/labicquette/.local/lib/python3.10/site-packages/transformers/pipelines/base.py", line 269, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
  File "/home/labicquette/.local/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py", line 566, in from_pretrained
    return model_class.from_pretrained(
  File "/home/labicquette/.local/lib/python3.10/site-packages/transformers/modeling_utils.py", line 3480, in from_pretrained
    ) = cls._load_pretrained_model(
  File "/home/labicquette/.local/lib/python3.10/site-packages/transformers/modeling_utils.py", line 3601, in _lo

In [ ]:
"""llm = HuggingFaceHub.(
    repo_id=repo_id, model_kwargs={"max_length":200, 
                                   "do_sample":True,
                                    "top_k":10,
                                    "num_return_sequences":1,
                                    "eos_token_id":tokenizer.eos_token_id}
)
llm = HuggingFacePipeline.from_model_id(
    model_id="falcon-7b-instruct",task="text-generation", device_map="auto", model_kwargs={"max_length":200, 
                                   "do_sample":True,
                                    "top_k":10,
                                    "num_return_sequences":1,
                                    "eos_token_id":tokenizer.eos_token_id}
)"""
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=pipeline)

In [ ]:
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [ ]:
agent_executor = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,handle_parsing_errors=True, verbose=True
)

In [ ]:
agent_executor.invoke(
    {
        "input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
    }
)



> Entering new AgentExecutor chain...
 She is a celebrity; use the celebrity rule.
Action: Google search
Action Input: <insert search query with a celebrity>
Observation: Google search is not a valid tool, try one of [Search, Calculator].
Thought: I can find a celebrity's age easily, use it is a celebrity rule.
Action: (1) Use a search engine
Input: (2) The input
Observation(3)):(4
<enter>
Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought: 
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought: 
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought: Invalid format: Missing 'Observation:' after 'Input:'
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought: Invalid format
<noinput>
As an AI language model, I cannot access the search engine or a calculator. However, you have the correct formatting and can use the following to get the result for the calculation: 


1. Who is Leo DiCaprio's girlfriend? 

2. Wha

/home/labicquette/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 519, but `max_length` is set to 500. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


 In
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:

/home/labicquette/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 540, but `max_length` is set to 500. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


 
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:

...

/home/labicquette/.local/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/labicquette/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 666, but `max_length` is set to 500. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


 Missing
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:

/home/labicquette/.local/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/labicquette/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 687, but `max_length` is set to 500. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


 Invalid
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:

> Finished chain.


{'input': "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?",
 'output': 'Agent stopped due to iteration limit or time limit.'}